In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
Y_train=train["label"].values

In [4]:
X_train = train.loc[:,'pixel0':'pixel783'].values
X_train = np.array(X_train)
X_train = X_train.reshape([-1,28,28,1])
Y_train = Y_train.reshape([-1,1])

In [5]:
X_test = test.loc[:,'pixel0':'pixel783'].values
X_test = np.array(X_test)
X_test = X_test.reshape([-1,28,28,1])

In [6]:
X_train=X_train/255
X_test=X_test/255
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 3)

In [7]:
Y_train=to_categorical(Y_train)
Y_val=to_categorical(Y_val)
print(Y_train.shape)
print(Y_val.shape)

(37800, 10)
(4200, 10)


In [8]:
inputs=Input(shape=(28,28,1))
conv0 = Conv2D(64, 3, padding = 'same', activation = 'tanh')(inputs)
conv1 = Conv2D(64, 3, padding='same', activation='tanh')(conv0)
conv2 = Conv2D(128, 3, padding='same', activation='tanh')(conv1)
pool2 = MaxPooling2D((2,2))(conv2)
conv3 = Conv2D(128, 3, padding='same', activation='tanh')(pool2)
conv4 = Conv2D(256, 5, padding='same', activation='tanh')(conv3)
pool4 = MaxPooling2D((2,2))(conv4)
conv5 = Conv2D(256, 5, padding='same', activation='tanh')(pool4)
flat = Flatten()(conv5)
dense0 = Dense(512, activation='relu')(flat)
dense1 = Dense(128, activation='relu')(dense0)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(10, activation='softmax')(dense2)

model = Model(inputs,dense3)
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 256)      

In [9]:
model.compile(Adam(lr = 10e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, Y_train, epochs = 20, batch_size = 200, validation_data = (X_val, Y_val), verbose = 1)

Epoch 1/20
189/189 [==============================] - 8s 43ms/step - loss: 0.3690 - accuracy: 0.9024 - val_loss: 0.1038 - val_accuracy: 0.9683
Epoch 2/20
189/189 [==============================] - 8s 42ms/step - loss: 0.0818 - accuracy: 0.9743 - val_loss: 0.0955 - val_accuracy: 0.9738
Epoch 3/20
189/189 [==============================] - 8s 42ms/step - loss: 0.0599 - accuracy: 0.9813 - val_loss: 0.0681 - val_accuracy: 0.9800
Epoch 4/20
189/189 [==============================] - 8s 42ms/step - loss: 0.0485 - accuracy: 0.9855 - val_loss: 0.0820 - val_accuracy: 0.9774
Epoch 5/20
189/189 [==============================] - 8s 42ms/step - loss: 0.0528 - accuracy: 0.9843 - val_loss: 0.1037 - val_accuracy: 0.9740
Epoch 6/20
189/189 [==============================] - 8s 42ms/step - loss: 0.0469 - accuracy: 0.9860 - val_loss: 0.1017 - val_accuracy: 0.9769
Epoch 7/20
189/189 [==============================] - 8s 43ms/step - loss: 0.0347 - accuracy: 0.9897 - val_loss: 0.0682 - val_accuracy: 0.9843

In [10]:
sol = model.predict(X_test, verbose = 1)

875/875 [==============================] - 3s 4ms/step


In [11]:
sol = np.argmax(sol, axis =1)

In [12]:
print(sol)

[2 0 9 ... 3 9 2]


In [13]:
sol = pd.Series(sol, name = 'Label')
index = pd.Series(range(1,28001), name = 'ImageID')
submission = pd.concat([index, sol], axis = 1)

In [14]:
submission.to_csv("submission.csv",index=False)